In [1]:
#| default_exp core

In [2]:
#| hide
from nbdev.showdoc import *

# Dependencies

In [37]:
#| export
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from pydantic_ai import Agent, ModelRetry, RunContext
from pydantic_ai.models import KnownModelName
from typing import Optional, Union
from functools import wraps

load_dotenv()

# Enable async/await in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Agent setup

System prompt


In [4]:
#| export
from datetime import date
system_prompt = f"""
You are a helpful assistant that operates in a Jupyter notebook.
Your regular text responses are rendered as cell output.
You can create new cells, edit existing cells, and run code.
You can also use tools to help you with your tasks.
Today's date is {date.today().strftime('%Y-%m-%d')}.
"""

In [5]:
#| export
from typing import cast
model = cast(KnownModelName, os.getenv('PYDANTIC_AI_MODEL', 'openai:gpt-4o'))
print(f'PydanticAI is using model: {model}')
notebook_agent = Agent(model, system_prompt=system_prompt)

PydanticAI is using model: openai:gpt-4o


In [39]:
#| export
_current_agent: Optional[Agent] = None

def get_current_agent() -> Agent:
    """Get the current agent, falling back to default notebook_agent if none set."""
    global _current_agent, notebook_agent
    return _current_agent or notebook_agent


### Adding cell creation tool

In [ ]:
#| export
from IPython.display import display, Markdown
from typing import Literal

@notebook_agent.tool
def create_cell(ctx: RunContext[str], content: str, cell_type: Literal['code', 'markdown'] = 'code') -> str:
    """Create a new cell in the notebook with the specified content.
    
    Args:
        content: The content to put in the new cell
        cell_type: Type of cell to create ('code' or 'markdown')
    
    Returns:
        A confirmation message
    """
    try:    
        ipython = get_ipython()
    except NameError:
        return "Error: Not running in IPython/Jupyter environment"
    
    # Display the content immediately
    if cell_type == 'code':
        # Set up the next cell with the content
        ipython.set_next_input(content)
    else:
        display(Markdown(content))
    
    return f"Created new {cell_type}  with content: {content}"

Always set custom agents to have create_cell tool

In [43]:
#| export
def set_agent(agent: Agent) -> Agent:
    """Set a custom agent for the notebook.
    
    Args:
        agent: PydanticAI agent instance
        system_prompt: Optional custom system prompt
        
    Returns:
        Configured agent with required tools
    """
    global _current_agent
        
    # Always ensure create_cell tool is available
    if 'create_cell' not in agent.function_tools:
        @agent.tool
        @wraps(create_cell)  # Preserve the original docstring
        def create_cell(ctx: RunContext[str], content: str, 
                       cell_type: Literal['code', 'markdown'] = 'code') -> str:
            return create_cell.original_func(ctx, content, cell_type)
    
    _current_agent = agent
    return agent

Tool testing

In [7]:
result = notebook_agent.run_sync('can you create me a simple hello world functio in a new celland make sure I can run it right away to see the output?')
Markdown(result.data)

I've created a new code cell with a simple "Hello, World!" function. You can run the cell to see the output.

In [8]:
def hello_world():
    print('Hello, World!')

# Call the function to display the output
hello_world()

Hello, World!


In [9]:
result = notebook_agent.run_sync("Create a function that calculates the factorial of a number with input validation")
Markdown(result.data)

I have created a function named `factorial` in a new code cell. This function calculates the factorial of a number and includes input validation to ensure that the input is a non-negative integer. You can run that cell to define the function in the notebook's environment.

In [10]:
def factorial(n):
    """
    Calculate the factorial of a non-negative integer n.

    Parameters:
    n (int): A non-negative integer whose factorial is to be computed

    Returns:
    int: Factorial of the input number n
    """
    # Input validation
    if not isinstance(n, int):
        raise TypeError("Input must be an integer")
    if n < 0:
        raise ValueError("Input must be a non-negative integer")

    # Base case: factorial of 0 is 1
    if n == 0:
        return 1

    # Recursive case: n! = n * (n-1)!
    return n * factorial(n - 1)

Testing agent with history

In [11]:
result = notebook_agent.run_sync('So what you just made for me here?', message_history=result.new_messages())
Markdown(result.data)

I've created a Python function called `factorial` for you. This function calculates the factorial of a given non-negative integer. Here's a brief overview of its features:

- **Input Validation**: It checks whether the input is a non-negative integer. If the input is not an integer, it raises a `TypeError`. If the input is a negative integer, it raises a `ValueError`.
- **Factorial Calculation**: It calculates the factorial using a recursive approach. The base case is that the factorial of 0 is 1, and for other numbers, it uses the recursive definition: \( n! = n \times (n-1)! \).

To make use of this function, you would call it with an integer argument, like so:

```python
factorial(5)
```

This would return `120`, which is the factorial of 5. You can test this function by running the code cell I created.

In [12]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content="\nYou are a helpful assistant that operates in a Jupyter notebook.\nYour regular text responses are rendered as cell output.\nYou can create new cells, edit existing cells, and run code.\nYou can also use tools to help you with your tasks.\nToday's date is 2024-12-31.\n", part_kind='system-prompt'), UserPromptPart(content='Create a function that calculates the factorial of a number with input validation', timestamp=datetime.datetime(2024, 12, 31, 8, 8, 34, 458473, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='create_cell', args=ArgsJson(args_json='{"content":"def factorial(n):\\n    \\"\\"\\"\\n    Calculate the factorial of a non-negative integer n.\\n\\n    Parameters:\\n    n (int): A non-negative integer whose factorial is to be computed\\n\\n    Returns:\\n    int: Factorial of the input number n\\n    \\"\\"\\"\\n    # Input validation\\n    if not isinstance(

### Adding notebook history

In [14]:
#| export
import os
import json
from pathlib import Path
from typing import Optional, Dict, Any

# Cache for notebook data
_notebook_cache: Dict[str, Any] = {}

def find_current_notebook() -> Optional[dict]:
    """Find and cache the current notebook data.
    
    Returns:
        Dict containing notebook data or None if not found
    """
    global _notebook_cache
    
    try:
        ipython = get_ipython()
        if not ipython:
            return None
            
        # Get current cell content to identify the notebook
        current_cell = ipython.get_parent()['content']['code']
        
        # Check if we already found the notebook
        if 'notebook' in _notebook_cache:
            # Verify it's still the correct notebook by checking the current cell
            notebook = _notebook_cache['notebook']
            for cell in notebook['cells']:
                if (cell['cell_type'] == 'code' and 
                    ''.join(cell['source']) == current_cell):
                    return notebook
        
        # If not in cache or cache is invalid, search for the notebook
        current_dir = Path.cwd()
        notebook_files = list(current_dir.glob("*.ipynb"))
        
        for nb_file in notebook_files:
            try:
                with open(nb_file) as f:
                    notebook = json.load(f)
                    for cell in notebook['cells']:
                        if (cell['cell_type'] == 'code' and 
                            ''.join(cell['source']) == current_cell):
                            # Found the notebook, cache it
                            _notebook_cache['notebook'] = notebook
                            _notebook_cache['file'] = nb_file
                            return notebook
            except Exception:
                continue
                
        return None
        
    except Exception as e:
        print(f"Error finding notebook: {e}")
        return None


In [27]:
#| export
def get_notebook_history(max_cells: int = 5) -> list:
    """Get the content of notebook cells between current and last prompt cell.
    
    Args:
        max_cells: Maximum number of previous cells to include
        
    Returns:
        List of previous cell contents
    """
    try:
        # Get the cached notebook or find it
        notebook = find_current_notebook()
        if not notebook:
            return []
            
        # Find current cell index
        current_cell = get_ipython().get_parent()['content']['code']
        cells = notebook['cells']
        current_idx = -1
        last_prompt_idx = -1
        
        # Find current cell and last prompt cell
        for idx, cell in enumerate(cells):
            source = ''.join(cell['source']) if isinstance(cell['source'], list) else cell['source']
            
            # Find current cell
            if current_idx == -1 and cell['cell_type'] == 'code' and source == current_cell:
                current_idx = idx
                
            # Find last prompt cell before current cell
            if idx < current_idx or current_idx == -1:  # Only look at cells before current
                if cell['cell_type'] == 'code' and source.strip().startswith('%%prompt'):
                    last_prompt_idx = idx
                
        if current_idx == -1:
            return []
            
        # Get cells between last prompt and current cell
        start_idx = last_prompt_idx + 1 if last_prompt_idx != -1 else max(0, current_idx - max_cells)
        history = []
        
        for idx in range(start_idx, current_idx):
            cell = cells[idx]            
            source = cell['source'] if isinstance(cell['source'], str) else ''.join(cell['source'])
            if 'outputs' in cell:
                outputs = cell['outputs'] if isinstance(cell['outputs'], str) else str(cell['outputs'])
            else:
                outputs = 'None'
            if not (source.strip().startswith('%%prompt') or outputs.strip().startswith('#|exclude') or outputs.strip().startswith('#| exclude')):
                history.append(f"Cell[{idx}]:\nSource:\n{source}\nOutputs:\n{outputs}")
        
        return history
        
    except Exception as e:
        print(f"Error getting notebook history: {e}")
        return []

Testing notebook history

In [28]:
nb_hist = get_notebook_history(max_cells=20)
nb_hist

['Cell[3]:\nSource:\n#| export\nimport os\nfrom dotenv import load_dotenv\nfrom pydantic import BaseModel, Field\nfrom pydantic_ai import Agent, ModelRetry, RunContext\nfrom pydantic_ai.models import KnownModelName\n\nload_dotenv()\n\n# Enable async/await in Jupyter\nimport nest_asyncio\nnest_asyncio.apply()\nOutputs:\n[]',
 'Cell[4]:\nSource:\n# Agent setup\nOutputs:\nNone',
 'Cell[5]:\nSource:\nSystem prompt\n\nOutputs:\nNone',
 'Cell[6]:\nSource:\n#| export\nfrom datetime import date\nsystem_prompt = f"""\nYou are a helpful assistant that operates in a Jupyter notebook.\nYour regular text responses are rendered as cell output.\nYou can create new cells, edit existing cells, and run code.\nYou can also use tools to help you with your tasks.\nToday\'s date is {date.today().strftime(\'%Y-%m-%d\')}.\n"""\nOutputs:\n[]',
 "Cell[7]:\nSource:\n#| export\nfrom typing import cast\nmodel = cast(KnownModelName, os.getenv('PYDANTIC_AI_MODEL', 'openai:gpt-4o'))\nprint(f'PydanticAI is using model

### Creating history-aware prompt

In [29]:
#| export
def create_history_aware_prompt(prompt: str, message_history: list = None, max_history: int = 5) -> tuple:
    """Create a prompt with notebook history context and message history.
    
    Args:
        prompt: The user's prompt
        message_history: Previous conversation messages from results.all_messages()
        max_history: Maximum number of previous cells to include
        
    Returns:
        Tuple of (enhanced prompt, combined message history)
    """
    try:
        ipython = get_ipython()
        if not ipython:
            return prompt, message_history
        
        # Get new cells using our optimized get_notebook_history
        new_cells = get_notebook_history(max_cells=max_history)
        
        if not new_cells and not message_history:
            return prompt, None
            
        # Create message history if none exists
        from pydantic_ai.messages import (
            ModelRequest, ModelResponse, 
            UserPromptPart, TextPart
        )
        
        messages = []
        
        # Add existing message history if provided
        if message_history:
            messages.extend(message_history)
        
        # Only add context message if we have new cells
        if new_cells:
            # Create context message with new cells
            history_content = "\n\n".join(new_cells)

            context_msg = ModelRequest(parts=[
                UserPromptPart(
                    content="Here is the context of new notebook cells that were added:\n" + history_content
                )
            ])
            
            # Create response acknowledging new context
            context_response = ModelResponse(parts=[
                TextPart(
                    content="I understand the new notebook context. How can I help?"
                )
            ])
            
            messages.extend([context_msg, context_response])
                
        return prompt, messages
        
    except Exception as e:
        print(f"Error creating history-aware prompt: {e}")
        return prompt, message_history

Testing history-aware prompt

In [30]:
create_history_aware_prompt('So what you just made for me here?', result.all_messages(), max_history=20)

('So what you just made for me here?',
 [ModelRequest(parts=[SystemPromptPart(content="\nYou are a helpful assistant that operates in a Jupyter notebook.\nYour regular text responses are rendered as cell output.\nYou can create new cells, edit existing cells, and run code.\nYou can also use tools to help you with your tasks.\nToday's date is 2024-12-31.\n", part_kind='system-prompt'), UserPromptPart(content='Create a function that calculates the factorial of a number with input validation', timestamp=datetime.datetime(2024, 12, 31, 8, 8, 34, 458473, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
  ModelResponse(parts=[ToolCallPart(tool_name='create_cell', args=ArgsJson(args_json='{"content":"def factorial(n):\\n    \\"\\"\\"\\n    Calculate the factorial of a non-negative integer n.\\n\\n    Parameters:\\n    n (int): A non-negative integer whose factorial is to be computed\\n\\n    Returns:\\n    int: Factorial of the input number n\\n    \\"\\"\\"\\n    # 

### Running agent with notebook history

In [31]:
#| export
from typing import Any
def run_with_history(agent: Agent, prompt: str, message_history: list = None, max_history: int = 5) -> Any:
    """Run the agent with notebook and conversation history context.
    
    Args:
        agent: The PydanticAI agent
        prompt: The user's prompt
        message_history: Previous conversation messages
        max_history: Maximum number of previous cells to include
        
    Returns:
        Agent run result
    """
    prompt, combined_history = create_history_aware_prompt(
        prompt, 
        message_history=message_history, 
        max_history=max_history
    )
    return agent.run_sync(prompt, message_history=combined_history)

Testing run_with_history

In [32]:
result = run_with_history(notebook_agent, 'So what is going on in this notebook?',result.all_messages(), max_history=20)
Markdown(result.data)


In this notebook, it seems you are exploring the use of a Jupyter notebook environment with some form of automation or tooling integration, likely involving the `pydantic_ai` library and custom functions to enhance interaction. Here's a breakdown of what's happening across the cells:

1. **Tool and Function Creation**: There are cells involving tool testing and function creation, such as a `hello_world` function and a `factorial` function with input validation. These are basic Python functions to demonstrate functionality and validate input for certain operations.

2. **Notebook Management Code**: Functions like `find_current_notebook` and `get_notebook_history` are used to identify and manage the notebook state, potentially to track changes or maintain state between interactions.

3. **Prompt and Response Handling**: Functions like `create_history_aware_prompt` are designed to enhance prompt interactions by incorporating notebook history and message history. This might be aimed at creating a more context-aware virtual assistant or agent within the notebook.

4. **Integration with External Agent**: There is mention of a `notebook_agent`, which appears to be an instance of a virtual assistant (possibly powered by AI) that interacts with the user through the notebook. This involves handling user prompts, generating responses, and creating new cells in the notebook.

5. **Testing and Debugging**: There are various points in the notebook where functions and features are being tested, like `run_with_history` which combines notebook context with conversational history to perform operations.

Overall, the notebook is set up to experiment with automated interactions using an AI-powered agent, managing notebooks dynamically, and running code through direct and possibly conversational interfaces. If there's anything specific you need help with, let me know!

In [33]:
result.all_messages()

[ModelRequest(parts=[SystemPromptPart(content="\nYou are a helpful assistant that operates in a Jupyter notebook.\nYour regular text responses are rendered as cell output.\nYou can create new cells, edit existing cells, and run code.\nYou can also use tools to help you with your tasks.\nToday's date is 2024-12-31.\n", part_kind='system-prompt'), UserPromptPart(content='Create a function that calculates the factorial of a number with input validation', timestamp=datetime.datetime(2024, 12, 31, 8, 8, 34, 458473, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'),
 ModelResponse(parts=[ToolCallPart(tool_name='create_cell', args=ArgsJson(args_json='{"content":"def factorial(n):\\n    \\"\\"\\"\\n    Calculate the factorial of a non-negative integer n.\\n\\n    Parameters:\\n    n (int): A non-negative integer whose factorial is to be computed\\n\\n    Returns:\\n    int: Factorial of the input number n\\n    \\"\\"\\"\\n    # Input validation\\n    if not isinstance(

### Creating prompt cell magic

In [44]:
#| export
from IPython.core.magic import register_cell_magic

#| export
@register_cell_magic
def prompt(line, cell):
    """Cell magic to create prompt cells that interact with the AI agent."""
    try:
        # Get the last result's message history if it exists
        message_history = None
        if 'last_prompt_result' in get_ipython().user_ns:
            last_result = get_ipython().user_ns['last_prompt_result']
            if hasattr(last_result, 'all_messages'):
                message_history = last_result.all_messages()
        
        # Use get_current_agent() instead of notebook_agent directly
        agent = get_current_agent()
        
        # Run the prompt through our agent with history context
        result = run_with_history(
            agent, 
            cell.strip(), 
            message_history=message_history
        )
        
        # Store the result for next time
        get_ipython().user_ns['last_prompt_result'] = result
        
        return Markdown(result.data)
    except Exception as e:
        return f"Error processing prompt: {str(e)}"

In [65]:
%%prompt
what this notebook is all about?

Error creating history-aware prompt: cannot import name 'ModelRequest' from 'pydantic_ai.messages' (/home/ndendic/WebDev/FH_SQLModel/.venv/lib/python3.12/site-packages/pydantic_ai/messages.py)


This notebook currently consists of a single interaction where you're asking about the notebook's purpose, but it doesn't have any specific content or code yet related to a particular topic. If you want to add specific content or have particular instructions or tasks you'd like to perform in the notebook, please let me know!

In [66]:
last_prompt_result.all_messages()

[SystemPrompt(content="\nYou are a helpful assistant that operates in a Jupyter notebook.\nYour regular text responses are rendered as cell output.\nYou can create new cells, edit existing cells, and run code.\nYou can also use tools to help you with your tasks.\nToday's date is 2024-12-30.\n", role='system'),
 UserPrompt(content='what this notebook is all about?', timestamp=datetime.datetime(2024, 12, 30, 16, 22, 54, 48018, tzinfo=datetime.timezone.utc), role='user'),
 ModelTextResponse(content="This notebook currently consists of a single interaction where you're asking about the notebook's purpose, but it doesn't have any specific content or code yet related to a particular topic. If you want to add specific content or have particular instructions or tasks you'd like to perform in the notebook, please let me know!", timestamp=datetime.datetime(2024, 12, 30, 16, 22, 54, tzinfo=datetime.timezone.utc), role='model-text-response')]

In [90]:
def last_prompt_result_data():
    return last_prompt_result.data
last_prompt_result_data()

"This notebook currently consists of a single interaction where you're asking about the notebook's purpose, but it doesn't have any specific content or code yet related to a particular topic. If you want to add specific content or have particular instructions or tasks you'd like to perform in the notebook, please let me know!"

In [67]:
def all_messages():
    return last_prompt_result.all_messages()
all_messages()

[SystemPrompt(content="\nYou are a helpful assistant that operates in a Jupyter notebook.\nYour regular text responses are rendered as cell output.\nYou can create new cells, edit existing cells, and run code.\nYou can also use tools to help you with your tasks.\nToday's date is 2024-12-30.\n", role='system'),
 UserPrompt(content='what this notebook is all about?', timestamp=datetime.datetime(2024, 12, 30, 16, 22, 54, 48018, tzinfo=datetime.timezone.utc), role='user'),
 ModelTextResponse(content="This notebook currently consists of a single interaction where you're asking about the notebook's purpose, but it doesn't have any specific content or code yet related to a particular topic. If you want to add specific content or have particular instructions or tasks you'd like to perform in the notebook, please let me know!", timestamp=datetime.datetime(2024, 12, 30, 16, 22, 54, tzinfo=datetime.timezone.utc), role='model-text-response')]

In [69]:
%%prompt
tell me what have I added last to this notebook?

Error creating history-aware prompt: cannot import name 'ModelRequest' from 'pydantic_ai.messages' (/home/ndendic/WebDev/FH_SQLModel/.venv/lib/python3.12/site-packages/pydantic_ai/messages.py)


The last addition to this notebook was your question, "what this notebook is all about?" There haven't been any other specific entries, code, or content added besides our interaction. If you want to include more content or perform tasks, feel free to specify!

In [69]:
all_messages()

[SystemPrompt(content="\nYou are a helpful assistant that operates in a Jupyter notebook.\nYour regular text responses are rendered as cell output.\nYou can create new cells, edit existing cells, and run code.\nYou can also use tools to help you with your tasks.\nToday's date is 2024-12-30.\n", role='system'),
 UserPrompt(content='what this notebook is all about?', timestamp=datetime.datetime(2024, 12, 30, 16, 22, 54, 48018, tzinfo=datetime.timezone.utc), role='user'),
 ModelTextResponse(content="This notebook currently consists of a single interaction where you're asking about the notebook's purpose, but it doesn't have any specific content or code yet related to a particular topic. If you want to add specific content or have particular instructions or tasks you'd like to perform in the notebook, please let me know!", timestamp=datetime.datetime(2024, 12, 30, 16, 22, 54, tzinfo=datetime.timezone.utc), role='model-text-response'),
 UserPrompt(content='tell me what have I added last to 

In [46]:
#| hide
import nbdev; nbdev.nbdev_export()